In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from tensorflow import keras
import tensorflow as tf

In [19]:
sheakespeare_url = "https://homl.info/shakespeare"

filepath = keras.utils.get_file("shakespeare.txt", sheakespeare_url)
with open(filepath) as f:
    sheakespeare_text = f.read()

In [20]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(sheakespeare_text)

In [21]:
seq = tokenizer.texts_to_sequences(["First"])
print(seq)

tokenizer.sequences_to_texts(seq)

[[20, 6, 9, 8, 3]]


['f i r s t']

In [22]:
max_id = len(tokenizer.word_index)
print(max_id)

39


In [23]:
dataset_size = tokenizer.document_count
dataset_size

1115394

In [24]:
[encoded] = np.array(tokenizer.texts_to_sequences([sheakespeare_text])) - 1
print(encoded)

[19  5  8 ... 20 26 10]


In [25]:
train_size = int(0.9 * dataset_size)
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [26]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [27]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [28]:
batch_size = 32
dataset = dataset.shuffle(10_000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [29]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [30]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [31]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")


In [35]:
history = model.fit(dataset, epochs=1)

     14/Unknown - 5s 283ms/step - loss: 1.7770

KeyboardInterrupt: 

In [36]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)



In [37]:
X_new = preprocess(["How are yo"])
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'u'

In [38]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [39]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [41]:
complete_text("t", temperature=0.2)

1/1 [==============================] - 0s 33ms/step


'the come,\nthe can the come sine and the comenius an'